# Aim
Create refrence archor images for age prediction using Twin Network.

In [162]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
import cv2
import glob
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import shutil
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_io as tfio
from tensorflow.keras import applications, layers, models

import os
import random

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


from twinnet_tools.tnconfig import ProjectConfig

config = ProjectConfig("twinnet_config")
dir_root_scripts = config.json["dir_scripts"]


import glob
import matplotlib as mpl
import pandas as pd
from pathlib import Path
import sys

sys.path.append(dir_root_scripts)
from twinnet_tools.tngeneral import TNToolsGeneral
from twinnet_tools.tninference import TNToolsEmbeddings
from twinnet_tools.tninference import TNToolsSimilarities
from twinnet_tools.tnmodel import TNToolsNetwork
from twinnet_tools.tnplot import TNToolsPlot

# Import tools

tools_general = TNToolsGeneral()
tools_embeddings = TNToolsEmbeddings(size_img=224,
                                     size_img_min=250)
tools_model = TNToolsNetwork()
tools_similarities = TNToolsSimilarities()
tools_plot = TNToolsPlot()

ProjectConfig: /home/hmorales/WorkSpace/Development/TwinNetwork/CodeGitHub/TwinNet-main/code/Scripts/twinnet_config/Linux/config.json


In [164]:
new_rc_params = {'text.usetex': False,
                'svg.fonttype': 'none',
                'lines.linewidth': 1}

mpl.rcParams.update(new_rc_params)

random.seed(1) # for reproducibility 

# 1. Paths

In [165]:
dir_src_imgs = "/media/hmorales/Skynet/TwinNet/temperature_zFish/trainData/zfish_285C/allEmbryos/"
dir_dst = "/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/"
relative_path = "./images_jsons_reference_zfish/images_reference/"
Number_anchors = 15

Create output directories

In [166]:
def create_folder(subfolder_path, folder_name):
    # Join the subfolder path and the folder name
    folder_path = os.path.join(subfolder_path, folder_name)

    try:
        # Create the folder
        os.makedirs(folder_path)
        print(f"Folder '{folder_name}' created successfully in '{subfolder_path}'!")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists in '{subfolder_path}'!")
        
def get_time_stamp(file_name):
    parts = file_name.split('--')
    time_stamp = parts[2].split('LO')[1]
    return time_stamp     

def get_images_by_subfolder(folder):
 
    embryoIds = sorted([folder for folder in os.listdir(dir_src_imgs) if os.path.isdir(os.path.join(dir_src_imgs, folder))])
    image_lists = []
 
    for subfolder in embryoIds:
        subfolder_path = os.path.join(folder, subfolder)
        tif_files = sorted([file for file in os.listdir(subfolder_path) if file.endswith('.tif')])
        image_lists.append(tif_files)

        
    return image_lists


def get_images_paths_by_subfolder(folder):
    
    embryoIds = sorted([folder for folder in os.listdir(dir_src_imgs) if os.path.isdir(os.path.join(dir_src_imgs, folder))])
    image_lists = []
 
    for subfolder in embryoIds:
        subfolder_path = os.path.join(folder, subfolder)
        tif_files = sorted([os.path.join(subfolder_path, file) for file in os.listdir(subfolder_path) if file.endswith('.tif')])
        image_lists.append(tif_files)

    return image_lists
    
    


def save_json(data, output_file):
    """Save json file."""
    with open(output_file, 'w') as file:
        json.dump(data, file, separators=(',', ':'))


In [167]:
# Cerate output directories

create_folder(dir_dst, 'images_reference')
images_reference_path = os.path.join(dir_dst, 'images_reference')

path_out = [None]*Number_anchors
# Iterate over the range from 1 to Number_anchors
for i in range(1, Number_anchors+1):
    i_padded = str(i).zfill(len(str(Number_anchors)))
    imgs_anchor_name = 'imgs_anchor_{}'.format(i_padded)
    create_folder(images_reference_path, imgs_anchor_name)
    path_out[i-1] = os.path.join(images_reference_path, imgs_anchor_name)
                               

Folder 'images_reference' created successfully in '/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/'!
Folder 'imgs_anchor_01' created successfully in '/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/images_reference'!
Folder 'imgs_anchor_02' created successfully in '/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/images_reference'!
Folder 'imgs_anchor_03' created successfully in '/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/images_reference'!
Folder 'imgs_anchor_04' created successfully in '/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/images_reference'!
Folder 'imgs_anchor_05' created successfully in '/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/images_reference'!
Folder 'imgs_anchor_06' created successfully in '/media/hmorales/Skynet/TwinNet/temperature_zFish/images_jsons_reference_zfish/images_refe

# 2. Choose rand images from all avaiable embryos

In [168]:
# Get avaible embryos and images
embryoIds = sorted([folder for folder in os.listdir(dir_src_imgs) if os.path.isdir(os.path.join(dir_src_imgs, folder))])
images_by_embryo = get_images_by_subfolder(dir_src_imgs)
images_paths_by_embryo = get_images_paths_by_subfolder(dir_src_imgs)
time_stamps = [get_time_stamp(file_name) for file_name in images_by_embryo[0]]   

for i, image_list in enumerate(images_by_embryo, 1):
    print(f"Subfolder {embryoIds[i-1]} contains : {len(image_list)} images")
    

# Get images N-plets and paths

# Create lists to store the paths
N_embryos = len(embryoIds) 
N_times = len(time_stamps)

paths_anchor = [None] * Number_anchors

for i in range(1, Number_anchors+1):
    i_padded = str(i).zfill(len(str(Number_anchors)))
    imgs_anchor_name = 'imgs_anchor_{}'.format(i_padded) + '/'
    paths_anchor[i-1] = [ os.path.join(relative_path, imgs_anchor_name)] * N_times



Subfolder E001 contains : 720 images
Subfolder E006 contains : 720 images
Subfolder E011 contains : 720 images
Subfolder E016 contains : 720 images
Subfolder E021 contains : 720 images
Subfolder E026 contains : 720 images
Subfolder E032 contains : 720 images
Subfolder E037 contains : 720 images
Subfolder E043 contains : 720 images
Subfolder E048 contains : 720 images
Subfolder E053 contains : 720 images
Subfolder E058 contains : 720 images
Subfolder E066 contains : 720 images
Subfolder E071 contains : 720 images
Subfolder E076 contains : 720 images
Subfolder E082 contains : 720 images
Subfolder E087 contains : 720 images
Subfolder E092 contains : 720 images
Subfolder E097 contains : 720 images
Subfolder E103 contains : 720 images
Subfolder E108 contains : 720 images
Subfolder E113 contains : 720 images
Subfolder E118 contains : 720 images
Subfolder E123 contains : 720 images
Subfolder E129 contains : 720 images
Subfolder E135 contains : 720 images


In [169]:
# get a triplet for each time point
for counter, time_stamp in enumerate(time_stamps, 1):
    random_Nplet = []
    while len(random_Nplet) < Number_anchors:
        num = random.randint(0, N_embryos-1)
        if num not in random_Nplet:
            random_Nplet.append(num)
    print(f"Counter: {counter}, Time Stamp: {time_stamp}, Random Nplet: {random_Nplet}")
    
    for i in range(0, Number_anchors):
        # check file consistency
        imgName = images_by_embryo[random_Nplet[i]][counter-1]
        if get_time_stamp(imgName) == time_stamp:
            # Store relative paths
            paths_anchor[i][counter-1] = os.path.join(paths_anchor[i][counter-1], imgName)
            #Copy images
            img_path = images_paths_by_embryo[random_Nplet[i]][counter-1]
            shutil.copyfile(img_path, os.path.join(path_out[i], imgName))      
        else:
            print(f"ERROR '{imgName} ' does not match time stamp '{time_stamp}'!")
            
    

# Save json files

for i in range(1, Number_anchors+1):
    i_padded = str(i).zfill(len(str(Number_anchors)))
    outname = 'paths_anchor_{}'.format(i_padded)+'.json'
    save_json(paths_anchor[i-1], os.path.join(dir_dst, outname))
       

Counter: 1, Time Stamp: 001, Random Nplet: [4, 18, 25, 24, 2, 8, 3, 15, 14, 20, 12, 6, 0, 13, 19]
Counter: 2, Time Stamp: 002, Random Nplet: [24, 0, 22, 14, 8, 23, 25, 7, 18, 3, 10, 20, 17, 12, 21]
Counter: 3, Time Stamp: 003, Random Nplet: [6, 13, 23, 0, 16, 7, 24, 14, 15, 17, 11, 21, 9, 20, 3]
Counter: 4, Time Stamp: 004, Random Nplet: [5, 20, 23, 9, 3, 10, 22, 16, 13, 21, 6, 18, 15, 12, 1]
Counter: 5, Time Stamp: 005, Random Nplet: [15, 7, 23, 25, 12, 13, 21, 5, 11, 17, 22, 24, 2, 14, 16]
Counter: 6, Time Stamp: 006, Random Nplet: [3, 24, 5, 16, 12, 11, 15, 23, 0, 1, 9, 22, 19, 18, 20]
Counter: 7, Time Stamp: 007, Random Nplet: [5, 16, 7, 0, 24, 6, 17, 12, 11, 18, 14, 8, 21, 19, 23]
Counter: 8, Time Stamp: 008, Random Nplet: [0, 12, 25, 23, 16, 4, 24, 17, 6, 13, 1, 15, 11, 18, 19]
Counter: 9, Time Stamp: 009, Random Nplet: [25, 19, 10, 14, 0, 7, 20, 5, 17, 18, 2, 8, 1, 21, 24]
Counter: 10, Time Stamp: 010, Random Nplet: [24, 8, 7, 3, 25, 19, 5, 11, 9, 2, 16, 21, 20, 22, 14]
Counter:

Counter: 84, Time Stamp: 084, Random Nplet: [4, 12, 22, 3, 23, 11, 2, 20, 17, 25, 16, 21, 18, 0, 19]
Counter: 85, Time Stamp: 085, Random Nplet: [9, 14, 21, 4, 2, 18, 6, 15, 25, 24, 10, 11, 5, 12, 3]
Counter: 86, Time Stamp: 086, Random Nplet: [19, 4, 8, 9, 21, 25, 20, 0, 17, 12, 23, 3, 14, 24, 13]
Counter: 87, Time Stamp: 087, Random Nplet: [19, 21, 13, 8, 11, 12, 14, 1, 3, 15, 24, 20, 22, 0, 25]
Counter: 88, Time Stamp: 088, Random Nplet: [1, 3, 18, 4, 16, 24, 11, 17, 8, 25, 20, 15, 22, 7, 19]
Counter: 89, Time Stamp: 089, Random Nplet: [7, 3, 17, 11, 5, 24, 1, 22, 10, 13, 23, 8, 21, 20, 19]
Counter: 90, Time Stamp: 090, Random Nplet: [13, 12, 11, 9, 24, 10, 14, 25, 22, 7, 20, 19, 16, 4, 1]
Counter: 91, Time Stamp: 091, Random Nplet: [10, 21, 3, 16, 5, 17, 20, 15, 24, 22, 18, 0, 6, 12, 7]
Counter: 92, Time Stamp: 092, Random Nplet: [3, 7, 10, 21, 25, 14, 23, 15, 11, 20, 24, 6, 13, 12, 17]
Counter: 93, Time Stamp: 093, Random Nplet: [3, 18, 15, 8, 4, 0, 12, 13, 25, 20, 2, 5, 14, 24, 2

Counter: 167, Time Stamp: 167, Random Nplet: [13, 4, 7, 22, 24, 21, 11, 10, 18, 23, 1, 16, 14, 3, 20]
Counter: 168, Time Stamp: 168, Random Nplet: [20, 3, 13, 4, 25, 0, 11, 9, 15, 2, 24, 18, 17, 19, 16]
Counter: 169, Time Stamp: 169, Random Nplet: [3, 4, 17, 21, 22, 12, 20, 19, 13, 7, 16, 15, 23, 10, 14]
Counter: 170, Time Stamp: 170, Random Nplet: [3, 2, 6, 18, 19, 22, 11, 20, 0, 16, 13, 7, 9, 15, 1]
Counter: 171, Time Stamp: 171, Random Nplet: [18, 13, 17, 9, 12, 20, 1, 21, 19, 0, 8, 15, 14, 7, 10]
Counter: 172, Time Stamp: 172, Random Nplet: [24, 15, 14, 17, 1, 8, 16, 5, 23, 22, 9, 18, 10, 21, 12]
Counter: 173, Time Stamp: 173, Random Nplet: [24, 21, 22, 13, 17, 19, 12, 15, 20, 7, 9, 0, 2, 4, 3]
Counter: 174, Time Stamp: 174, Random Nplet: [11, 8, 9, 25, 17, 4, 3, 16, 14, 1, 15, 23, 18, 10, 22]
Counter: 175, Time Stamp: 175, Random Nplet: [0, 17, 6, 24, 8, 19, 25, 2, 14, 9, 20, 23, 16, 22, 18]
Counter: 176, Time Stamp: 176, Random Nplet: [12, 3, 21, 10, 19, 20, 22, 18, 14, 2, 15, 16

Counter: 251, Time Stamp: 251, Random Nplet: [14, 25, 24, 5, 12, 20, 15, 18, 1, 16, 6, 21, 7, 4, 3]
Counter: 252, Time Stamp: 252, Random Nplet: [8, 17, 0, 11, 9, 6, 1, 21, 4, 2, 22, 5, 18, 13, 24]
Counter: 253, Time Stamp: 253, Random Nplet: [25, 4, 2, 6, 24, 5, 19, 13, 12, 17, 15, 7, 21, 22, 8]
Counter: 254, Time Stamp: 254, Random Nplet: [2, 24, 18, 11, 16, 8, 5, 21, 15, 13, 25, 17, 22, 7, 4]
Counter: 255, Time Stamp: 255, Random Nplet: [17, 18, 3, 25, 13, 23, 11, 7, 14, 21, 12, 10, 5, 1, 24]
Counter: 256, Time Stamp: 256, Random Nplet: [1, 11, 19, 14, 5, 21, 23, 18, 4, 6, 17, 15, 9, 7, 3]
Counter: 257, Time Stamp: 257, Random Nplet: [21, 2, 20, 7, 13, 16, 6, 9, 15, 22, 1, 12, 23, 19, 0]
Counter: 258, Time Stamp: 258, Random Nplet: [24, 14, 10, 13, 7, 20, 3, 5, 16, 1, 23, 12, 0, 15, 11]
Counter: 259, Time Stamp: 259, Random Nplet: [17, 13, 0, 20, 19, 21, 12, 7, 16, 1, 6, 10, 4, 14, 22]
Counter: 260, Time Stamp: 260, Random Nplet: [5, 14, 4, 8, 16, 12, 22, 2, 3, 15, 11, 6, 1, 13, 7]


Counter: 335, Time Stamp: 335, Random Nplet: [8, 24, 15, 22, 11, 5, 7, 1, 9, 20, 14, 17, 19, 0, 4]
Counter: 336, Time Stamp: 336, Random Nplet: [6, 5, 9, 2, 8, 12, 20, 25, 24, 18, 23, 14, 21, 11, 19]
Counter: 337, Time Stamp: 337, Random Nplet: [11, 7, 0, 25, 17, 19, 5, 14, 4, 9, 12, 10, 8, 6, 20]
Counter: 338, Time Stamp: 338, Random Nplet: [0, 1, 20, 14, 9, 7, 16, 24, 3, 23, 2, 19, 4, 6, 5]
Counter: 339, Time Stamp: 339, Random Nplet: [0, 13, 5, 14, 10, 1, 19, 16, 11, 22, 9, 18, 4, 17, 7]
Counter: 340, Time Stamp: 340, Random Nplet: [3, 14, 15, 7, 16, 10, 21, 9, 4, 8, 20, 13, 0, 11, 5]
Counter: 341, Time Stamp: 341, Random Nplet: [1, 10, 5, 0, 8, 7, 23, 9, 15, 19, 20, 18, 2, 6, 11]
Counter: 342, Time Stamp: 342, Random Nplet: [22, 23, 2, 25, 11, 4, 6, 14, 12, 19, 18, 7, 3, 9, 16]
Counter: 343, Time Stamp: 343, Random Nplet: [11, 2, 3, 18, 19, 7, 17, 9, 24, 20, 25, 5, 22, 13, 6]
Counter: 344, Time Stamp: 344, Random Nplet: [14, 4, 24, 6, 2, 13, 12, 18, 7, 9, 8, 19, 16, 23, 15]
Counter

Counter: 419, Time Stamp: 419, Random Nplet: [4, 2, 1, 3, 9, 10, 15, 18, 19, 0, 12, 17, 7, 23, 16]
Counter: 420, Time Stamp: 420, Random Nplet: [4, 17, 24, 3, 13, 12, 7, 0, 19, 18, 10, 25, 20, 21, 6]
Counter: 421, Time Stamp: 421, Random Nplet: [4, 13, 5, 3, 18, 23, 9, 11, 15, 24, 16, 8, 20, 22, 7]
Counter: 422, Time Stamp: 422, Random Nplet: [8, 2, 22, 16, 10, 7, 20, 25, 19, 17, 24, 5, 4, 1, 14]
Counter: 423, Time Stamp: 423, Random Nplet: [11, 4, 19, 7, 9, 23, 1, 13, 14, 15, 5, 17, 12, 25, 21]
Counter: 424, Time Stamp: 424, Random Nplet: [21, 15, 20, 23, 10, 9, 8, 5, 11, 16, 2, 24, 13, 7, 22]
Counter: 425, Time Stamp: 425, Random Nplet: [11, 3, 7, 22, 2, 21, 17, 9, 1, 18, 16, 0, 12, 5, 14]
Counter: 426, Time Stamp: 426, Random Nplet: [3, 6, 11, 2, 16, 4, 12, 5, 14, 19, 8, 22, 15, 25, 21]
Counter: 427, Time Stamp: 427, Random Nplet: [4, 3, 20, 2, 12, 5, 7, 25, 9, 1, 19, 6, 0, 22, 14]
Counter: 428, Time Stamp: 428, Random Nplet: [22, 6, 19, 12, 9, 4, 10, 11, 3, 25, 0, 13, 14, 23, 5]
Co

Counter: 503, Time Stamp: 503, Random Nplet: [5, 24, 22, 11, 13, 21, 8, 6, 12, 16, 7, 14, 4, 19, 9]
Counter: 504, Time Stamp: 504, Random Nplet: [18, 10, 5, 21, 17, 19, 9, 7, 25, 6, 1, 4, 0, 13, 11]
Counter: 505, Time Stamp: 505, Random Nplet: [22, 14, 4, 13, 6, 5, 17, 11, 23, 2, 8, 10, 1, 16, 9]
Counter: 506, Time Stamp: 506, Random Nplet: [21, 4, 23, 19, 15, 20, 1, 25, 14, 24, 13, 2, 0, 6, 11]
Counter: 507, Time Stamp: 507, Random Nplet: [6, 14, 1, 22, 7, 9, 17, 24, 5, 18, 11, 23, 4, 3, 10]
Counter: 508, Time Stamp: 508, Random Nplet: [5, 17, 25, 20, 14, 21, 15, 0, 2, 10, 18, 11, 13, 4, 22]
Counter: 509, Time Stamp: 509, Random Nplet: [3, 14, 21, 25, 10, 5, 20, 2, 12, 0, 16, 24, 19, 4, 9]
Counter: 510, Time Stamp: 510, Random Nplet: [25, 8, 9, 1, 15, 18, 4, 0, 19, 13, 7, 3, 23, 12, 21]
Counter: 511, Time Stamp: 511, Random Nplet: [17, 16, 0, 18, 9, 7, 22, 20, 12, 8, 11, 23, 6, 19, 13]
Counter: 512, Time Stamp: 512, Random Nplet: [6, 23, 14, 16, 24, 1, 4, 25, 17, 19, 8, 20, 3, 12, 15]

Counter: 587, Time Stamp: 587, Random Nplet: [18, 24, 20, 11, 2, 17, 14, 9, 15, 23, 6, 10, 12, 5, 8]
Counter: 588, Time Stamp: 588, Random Nplet: [17, 23, 4, 7, 5, 10, 21, 22, 0, 3, 6, 16, 1, 12, 24]
Counter: 589, Time Stamp: 589, Random Nplet: [4, 10, 15, 14, 3, 0, 12, 2, 23, 19, 16, 8, 9, 18, 21]
Counter: 590, Time Stamp: 590, Random Nplet: [22, 3, 18, 14, 2, 16, 24, 20, 0, 6, 11, 25, 21, 17, 9]
Counter: 591, Time Stamp: 591, Random Nplet: [20, 21, 0, 13, 17, 5, 19, 8, 23, 1, 18, 6, 22, 3, 11]
Counter: 592, Time Stamp: 592, Random Nplet: [1, 21, 22, 9, 3, 19, 5, 6, 0, 7, 13, 18, 4, 16, 14]
Counter: 593, Time Stamp: 593, Random Nplet: [15, 1, 4, 8, 14, 17, 24, 22, 11, 18, 10, 19, 21, 13, 7]
Counter: 594, Time Stamp: 594, Random Nplet: [23, 11, 14, 15, 0, 6, 10, 3, 1, 24, 22, 5, 8, 21, 17]
Counter: 595, Time Stamp: 595, Random Nplet: [7, 14, 22, 13, 4, 24, 5, 18, 25, 1, 16, 10, 15, 23, 21]
Counter: 596, Time Stamp: 596, Random Nplet: [22, 9, 17, 8, 16, 12, 7, 10, 18, 19, 2, 20, 15, 5, 

Counter: 670, Time Stamp: 670, Random Nplet: [15, 22, 25, 5, 18, 2, 4, 10, 21, 12, 7, 1, 19, 9, 6]
Counter: 671, Time Stamp: 671, Random Nplet: [11, 14, 23, 21, 6, 16, 3, 8, 13, 20, 9, 7, 4, 25, 0]
Counter: 672, Time Stamp: 672, Random Nplet: [3, 21, 25, 1, 24, 10, 16, 4, 19, 23, 14, 18, 8, 2, 7]
Counter: 673, Time Stamp: 673, Random Nplet: [9, 21, 13, 5, 20, 7, 4, 14, 15, 11, 0, 18, 10, 12, 1]
Counter: 674, Time Stamp: 674, Random Nplet: [11, 23, 7, 10, 24, 15, 0, 16, 2, 4, 1, 14, 22, 25, 8]
Counter: 675, Time Stamp: 675, Random Nplet: [7, 4, 8, 16, 23, 20, 11, 12, 15, 5, 25, 10, 17, 13, 6]
Counter: 676, Time Stamp: 676, Random Nplet: [8, 18, 12, 14, 17, 24, 2, 0, 6, 11, 19, 1, 5, 15, 20]
Counter: 677, Time Stamp: 677, Random Nplet: [7, 13, 15, 11, 23, 14, 0, 16, 24, 5, 20, 18, 3, 1, 12]
Counter: 678, Time Stamp: 678, Random Nplet: [20, 16, 1, 22, 6, 10, 4, 18, 21, 23, 12, 2, 24, 14, 8]
Counter: 679, Time Stamp: 679, Random Nplet: [23, 15, 11, 17, 22, 7, 1, 5, 25, 8, 4, 16, 10, 2, 6]
